# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


nltk.download(['wordnet', 'punkt', 'stopwords'])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')

df = pd.read_sql_table('InsertTableName',con=engine)
#df = pd.read_sql_table(table_name,con=engine)

df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']   # Target
Y =df.iloc[:,4:] #features

### 2. Write a tokenization function to process your text data

In [4]:
stop_words = stopwords.words("english")# Remove stop words# Lemmatization
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    # normalize case and remove punctuation
    text=re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    # tokenize text
    
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
#create pipeline
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#split data fram and fit by pipeline
X_train, X_test, Y_train, Y_test = train_test_split(X,Y , test_size = 0.3,random_state=42)
pipeline.fit(X_train,Y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
#predict data by pipeline
y_pred = pipeline.predict(X_test)

In [8]:
#scoreing resurt
def  score_precision_recall(Y_test,y_pred):
    """
    INPUT : model object, y_pred , Y_test 
     Output: reult of classification_report
     PROCESSING:
         1 - use and print result from classification_report
    """
    
    for i , colna in enumerate(Y_test):
        print('Column name: ',colna)
        print(classification_report(Y_test[colna],y_pred[:,i]))

In [9]:
score_precision_recall(Y_test,y_pred)

Column name:  related
             precision    recall  f1-score   support

          0       0.62      0.44      0.51      1873
          1       0.83      0.91      0.87      5934
          2       0.37      0.40      0.38        58

avg / total       0.78      0.79      0.78      7865

Column name:  request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6533
          1       0.79      0.43      0.56      1332

avg / total       0.88      0.88      0.87      7865

Column name:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

Column name:  aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      4646
          1       0.73      0.61      0.66      3219

avg / total       0.75      0.75      0.74      786

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
#have parameters and use GridSearchCV 
parameters = {'clf__estimator__n_estimators':[10,50],
             'vect__max_df':[ 1.0,0.5]}

cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=5, verbose=2)

cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=5,
       param_grid={'clf__estimator__n_estimators': [10, 50], 'vect__max_df': [1.0, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Fit the model for the second time
cv.fit(X_train,Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=1.0 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=1.0, total= 4.0min
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, total= 4.0min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, total= 4.0min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=10, vect__max_df=1.0, total= 4.0min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estima

In [ ]:
#make preaction and scoring
y_pred2 = cv.predict(X_test)
score_precision_recall(Y_test,y_pred2)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#repate process with  AdaBoostClassifier and change parameters
pipeline2  = pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])
parameters2 = {'clf__estimator__n_estimators':[10,80],
             'vect__max_df':[ 1.0,0.7]}
cv = GridSearchCV(pipeline2,param_grid=parameters2, n_jobs=5, verbose=2)


In [ ]:
#fit agein
cv.fit(X_train,Y_train)

In [ ]:
#make preaction and scoring
y_pred3 = cv.predict(X_test)
score_precision_recall(Y_test,y_pred3)

### 9. Export your model as a pickle file

In [ ]:
#save as  pickle file
import pickle
pickle.dump(cv,open('model_CV.pkl','wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import sys
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

nltk.download(['wordnet', 'punkt', 'stopwords'])


def load_data(database_filepath, table_name='InsertTableName'):
    """
    INPUT      : Takes a datafile name and relative path as parameter
        OUTPUT     : Pandas dataframe
       
        PROCESSING : 1. create a Database connection object    
                     2. Read SQL table into a pandas dataframe
                     3. retrun the dataframe
    """
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql_table(table_name,con=engine)
    X = df['message']   # Target
    Y =df.iloc[:,4:] #features
    
    return X,Y,Y.columns


stop_words = stopwords.words("english")# Remove stop words# Lemmatization
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    """
    INPUT   : Takes a sentences  
    OUTPUT  : list of tokens from the sentence 
       
    PROCESSING : parse the sentence with nltk word tokeniser
                 parse again the wordnet lemmatize
                 remove stop words
                 make the tokens into lower case and remove spaces
                 return the cleaned tokes as a list
    """
    # normalize case and remove punctuation
    text=re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    # tokenize text
    
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens


def build_model():
    """
    define pipeline useing :1-CountVectorizer
                            2-TfidfTransformer
                            3-MultiOutputClassifier-----AdaBoostClassifier
    define parameters
    define cv useing :GridSearchCV
    """
     
    pipeline = pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
            ])
    parameters = {'clf__estimator__n_estimators':[10,80],
             'vect__max_df':[ 1.0,0.7]}
    cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=5, verbose=2)   
    return cv


def evaluate_model(model, X_test, Y_test, category_names):
    """
     INPUT : model object, X_test , Y_test and  category names
     Output: reult of classification_report
     PROCESSING:
         1 - predict using model for test data 
         2 - use and print result from classification_report
    """
    y_pred = model.predict(X_test)
    for i , category_names in enumerate(Y_test):
        print('Column name: ',category_names)
        print(classification_report(Y_test[category_names],y_pred[:,i]))


def save_model(model, model_filepath):
    pickle.dump( model, open( model_filepath, "wb" ) )
    return None


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()